# Get data

In [ ]:
# importing needed libraries
import matplotlib.pyplot as plt
import numpy as np
import skimage
import util_affine

#### loading data
points_source = np.load('data/points_source.npy')
points_destination = np.load('data/points_destination.npy')



def build_target_vector(dst_points):
    # dst_points: array of shape (N, 2) with rows [x'_i, y'_i]
    return dst_points.flatten()

def build_affine_design_matrix(src_points):
    # src_points: array of shape (N, 2) with rows [x_i, y_i]
    x = src_points[:, 0]
    y = src_points[:, 1]

    rows = []
    for xi, yi in zip(x, y):
        # Row for x' equation
        rows.append([xi, yi, 1, 0, 0, 0])
        # Row for y' equation
        rows.append([0, 0, 0, xi, yi, 1])
    A = np.array(rows)
    return A

def estimate_affine_parameters(A, b):
    # Solve using least squares
    w = np.linalg.inv(A.T @ A) @ A.T @ b
    return w  # shape (6,)

def build_affine_matrix(w):
    T = np.array([
        [w[0], w[1], w[2]],
        [w[3], w[4], w[5]],
        [0,    0,    1   ]
    ])
    return T



### affine tranformation creation

def calc_affine(points_source, points_destination):
    """
    Estimate the affine transformation matrix using the corresponding points pairs
    
    Args:
        points_source: Points in the video 
        points_destination: Corresponding points in the map

    Returns:
        The affine matrix T
    """

    A = build_affine_design_matrix(points_source)
    b = build_target_vector(points_destination)
    w = estimate_affine_parameters(A, b)
    T = build_affine_matrix(w)

    print("Design matrix A:\n", A)
    print("\nTarget vector b:\n", b)
    print("\nParameters w:\n", w)

    return T


T = calc_affine(points_source, points_destination)
print('\nThe affine Trasformation Matrix:\n', T)


def load_data():
    """Loads the tracking data."""
    filename = "data/trackingdata.dat"
    data = np.loadtxt(filename)
    data = {"body": data[:, :4], "legs": data[:, 4:8], "all": data[:, 8:]}

    return data


### Load data 
data = load_data()


# Extract frames from video

In [ ]:
import cv2

def load_video_frames(video_path, max_frames=None):
    cap = cv2.VideoCapture(video_path)
    frames = []
    count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        # Convert from BGR (OpenCV default) to RGB (matplotlib expects RGB)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(frame)

        count += 1
        if max_frames and count >= max_frames:
            break

    cap.release()
    return frames

video_frames = load_video_frames("data/ITUStudent.mov")  # limit if needed
# print(video_frames)

# Export map trace as GIF

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np

def animate_map(tracking_data, T, fps=10):
    fig, ax = plt.subplots(figsize=(8,6))
    map_img = plt.imread("data/ITUMap.png")
    ax.imshow(map_img)
    ax.set_title("Projected center on ITU Map")

    xs, ys = [], []
    point, = ax.plot([], [], 'ro', label="current position")
    trace, = ax.plot([], [], 'g-', alpha=0.7, label="trajectory")

    n_frames = tracking_data['all'].shape[0]

    def update(i):
        # center of "all" bbox
        x1,y1,x2,y2 = tracking_data['all'][i]
        cx, cy = (x1+x2)/2, (y1+y2)/2

        # project to map
        proj = T @ np.array([cx, cy, 1])
        proj /= proj[2]

        xs.append(proj[0])
        ys.append(proj[1])

        point.set_data([proj[0]], [proj[1]])
        trace.set_data(xs, ys)
        return point, trace

    ani = animation.FuncAnimation(
        fig, update, frames=n_frames,
        interval=1000/fps, blit=True, repeat=False
    )
    plt.legend()
    plt.show()
    return ani

ani = animate_map(data, T, fps=15)

# # Save as mp4 (requires ffmpeg installed)
# ani.save("trajectory.mp4", writer="ffmpeg", fps=15)

# Or save as gif (requires imagemagick or pillow)
# ani.save("map_trajectory.gif", writer="pillow", fps=15)

# Export video and map trace as GIF

In [ ]:
import cv2
print("Frames:", cv2.VideoCapture("data/ITUStudent.mov").get(cv2.CAP_PROP_FRAME_COUNT))

import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.patches as patches
import numpy as np

def animate_video_and_map(video_frames, tracking_data, part, T, fps=10, outname="video_and_map.gif"):
    fig, ax = plt.subplots(1, 2, figsize=(14, 7))

    # --- Left: video plane ---
    im_video = ax[0].imshow(video_frames[0])
    ax[0].set_title("Video with bounding boxes")

    colors = {'body':'r','legs':'b','all':'g'}
    rects = {b_part: patches.Rectangle((0,0),1,1,fill=False,edgecolor=color,linewidth=2)
             for b_part,color in colors.items()}
    for rect in rects.values():
        ax[0].add_patch(rect)
    center_point, = ax[0].plot([], [], 'yo')

    # --- Right: map plane ---
    map_img = plt.imread("data/ITUMap.png")
    im_map = ax[1].imshow(map_img)
    ax[1].set_title("Projected center on ITU Map")
    proj_point, = ax[1].plot([], [], 'ro')
    trace, = ax[1].plot([], [], 'g-', alpha=0.7)

    xs, ys = [], []
    n_frames = min(len(video_frames), tracking_data['all'].shape[0])

    def update(i):
        # Update video frame
        im_video.set_data(video_frames[i])

        # Update bounding boxes
        for b_part, rect in rects.items():
            x1,y1,x2,y2 = tracking_data[b_part][i]
            rect.set_xy((x1,y1))
            rect.set_width(x2-x1)
            rect.set_height(y2-y1)

        # Center point
        x1,y1,x2,y2 = tracking_data['all'][i]
        cx, cy = (x1+x2)/2, (y1+y2)/2
        center_point.set_data([cx],[cy])

        # Project to map
        proj = T @ np.array([cx, cy, 1])
        proj /= proj[2]
        xs.append(proj[0]); ys.append(proj[1])
        proj_point.set_data([proj[0]], [proj[1]])
        trace.set_data(xs, ys)

        return [im_video, center_point, proj_point, trace] + list(rects.values())

    ani = animation.FuncAnimation(fig, update, frames=n_frames,
                                  interval=1000/fps, blit=True, repeat=False)

    # Save as GIF
    ani.save(outname, writer="pillow", fps=fps)
    # plt.close(fig)
    print(f"Saved combined animation to {outname}")

# animate_video_and_map(video_frames, data, part, T, fps=15, outname="combined_videoMap.gif")